In [7]:
"""
Script to calculate baseline (time 0) indicators for:
- the 5 boroughs with the highest household food insecurity
- the 5 boroughs with the lowest household food insecurity

Inputs
------
simulation_runs.xlsx             # SSD outputs per borough (NUM)
boroughs_name_NUM_match.xlsx     # mapping NUM -> borough name

Outputs
-------
table1.csv   # CSV with the values shown in Table 1
"""

import pandas as pd
import ast

# --------------------------------------------------------------------
# 1. Load data
# --------------------------------------------------------------------
# Adjust paths if needed (e.g. r"data/simulation_runs.xlsx")
sim_path = "simulation_runs.xlsx"
name_path = "boroughs_name_NUM_match.xlsx"

sim = pd.read_excel(sim_path)
names = pd.read_excel(name_path)

# merge to get borough names
df = sim.merge(names, on="NUM", how="left")

# --------------------------------------------------------------------
# 2. Helper: extract value at time 0 from dict-like columns
# --------------------------------------------------------------------
def get_time0(series):
    """
    series: a pandas Series where each cell is either
      - a Python dict, e.g. {'0.0': value0, '1.0': value1, ...}
      - or a string representation of such a dict

    returns: Series of the time 0 values.
    """
    values = []

    for x in series:
        if isinstance(x, dict):
            d = x
        elif isinstance(x, str) and x.startswith("{"):
            # safer than eval
            d = ast.literal_eval(x)
        else:
            values.append(pd.NA)
            continue

        # keys are usually '0.0', '1.0', ...
        # take the smallest key (time 0)
        try:
            # convert keys to float to sort by time
            key0 = sorted(d.keys(), key=lambda k: float(k))[0]
            values.append(d[key0])
        except Exception:
            values.append(pd.NA)

    return pd.Series(values, index=series.index)


# --------------------------------------------------------------------
# 3. Extract time-0 values for the variables used in Table 1
# --------------------------------------------------------------------
cols = {
    "HFI": "householdFoodInsecurity",
    "HFIcirc": "householdCircumstancesFoodInsecurity",
    "AvailHealthy": "availabilityOfHealthyFoodInLocalArea",
    "AccessHealthy":"accessibilityOfHealthyFoodInLocalArea",
    "StoreDensity": "supermarketAndHealthyFoodStoreDensity",
    "FastFoodDensity": "fastFoodOutletDensity",
    "ShelterLess30": "shelterCostsLessThan30PercentStock",
    "ShelterMore30": "shelterCostsMoreThan30PercentStock",
    "PTaccess": "accessibilityAndAffordabilityOfPublicTransport",
}

for new_name, original_col in cols.items():
    df[new_name] = get_time0(df[original_col])

# --------------------------------------------------------------------
# 4. Identify 5 highest and 5 lowest HFI boroughs at time 0
# --------------------------------------------------------------------
# sort by HFI
df_sorted = df.sort_values("HFI")

# 5 lowest HFI
lowest5 = df_sorted.head(18)

# 5 highest HFI
highest5 = df_sorted.tail(18)

# combine in one table
table1 = pd.concat([highest5, lowest5], axis=0)

# select and rename columns for readability
table1 = table1[["NUM", "NOM_y", "HFI", "HFIcirc",
                 "AvailHealthy", "AccessHealthy", "StoreDensity", "FastFoodDensity",
                 "ShelterLess30", "ShelterMore30", "PTaccess"]].copy()

# table1 = table1.rename(columns={
#     "NOM_y": "Borough",
#     "HFI": "HFI_t0",
#     "AvailHealthy": "Availability of healthy food in local area at t0",
#     "StoreDensity": "Supermarket and healthy food store density at t0",
#     "AccessHealthy": "Accessibility of healthy food in local area at t0",
#     "PTaccess": "Accessibility and affordability of public transport at t0",
#     "HHcirc": "Household circumstances food insecurity at t0",
# })

table1 = table1.rename(columns={
    "NOM_y": "Borough",
    "HFI": "HFI_t0",
    "HFIcirc": "HFIcirc_t0",
    "AvailHealthy": "AvailHealthy_t0",
    "AccessHealthy": "AccessHealthy_t0",
    "StoreDensity": "StoreDensity_t0",
    "FastFoodDensity": "FastFoodDensity_t0",
    "ShelterLess30": "ShelterLess30_t0",
    "ShelterMore30": "ShelterMore30_t0",
    "PTaccess": "PTaccess_t0",
})

# round for a nicer table (optional)
table1 = table1.round({
    "HFI_t0": 3,
    "HFIcirc_t0": 3,
    "AvailHealthy_t0": 3,
    "AccessHealthy_t0": 3,
    "StoreDensity_t0": 3,
    "FastFoodDensity_t0": 3,
    "ShelterLess30_t0": 3,
    "ShelterMore30_t0": 3,
    "PTaccess_t0": 3,
})

# sort so that highest-HFI rows appear first, then lowest
table1 = table1.sort_values("HFI_t0", ascending=False)

# --------------------------------------------------------------------
# 5. Save to CSV (and/or print)
# --------------------------------------------------------------------
# table1.to_csv("table_five.csv", index=False)
table1.to_excel('table_five.xlsx', index=False)
print(table1)


    NUM                                   Borough  HFI_t0  HFIcirc_t0  \
30   20                               Ville-Marie   0.218       0.096   
23   22                     Le Plateau-Mont-Royal   0.175       0.099   
4    76                   Sainte-Anne-de-Bellevue   0.169       0.084   
8     1                                    Dorval   0.165       0.081   
20    2                                Mont-Royal   0.160       0.083   
32    4                                 Westmount   0.157       0.087   
6    27       Côte-des-Neiges-Notre-Dame-de-Grâce   0.157       0.103   
11    3                                  Kirkland   0.156       0.064   
5    72                            Côte-Saint-Luc   0.154       0.091   
7    11                       Dollard-des-Ormeaux   0.149       0.074   
28   77                                Senneville   0.147       0.069   
14   15                             Saint-Laurent   0.145       0.092   
22    8                             Pointe-Claire  